# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: <b>LoRA</b>
* Model: <b>DistilBERT (distilbert-base-uncased)</b>
* Evaluation approach: <b>Huggingface Trainer with Evaluate</b>
* Fine-tuning dataset: <b>IMDB</b> dataset

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

#### Loading Model and Dataset

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("imdb")

MODEL_NAME = "distilbert-base-uncased"

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 126970.30 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [5]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


#### Tokenize the data

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [7]:
def tokenize_data(data):
    return tokenizer(data["text"], padding="max_length", truncation=True, max_length=256)

In [8]:
tokenized_datasets = dataset.map(tokenize_data, batched=True)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

Map: 100%|██████████| 50000/50000 [00:08<00:00, 5757.89 examples/s]


#### Evaluate the base model

In [9]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

In [10]:
# This function computes the metrics (accuracy here)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results/base_model",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
# Evaluate the base model with pre-trained weights
accuracy_before_tuning = trainer.evaluate()

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

#### Fine-Tuning the model using PEFT

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.1,
    bias="none",
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

#### Train the Light-weight model

In [13]:
training_args = TrainingArguments(
    output_dir="./results/light_weight_tuned_model",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

TrainOutput(global_step=126, training_loss=0.47251365298316594, metrics={'train_runtime': 825.1234, 'train_samples_per_second': 2.424, 'train_steps_per_second': 0.153, 'total_flos': 134739406848000.0, 'train_loss': 0.47251365298316594, 'epoch': 2.0})

In [14]:
# Save the peft model
peft_model.save_pretrained("./fine_tuned_imdb_distilbert-base-uncased")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
# Load the fine tuned peft model
from peft import AutoPeftModelForSequenceClassification

peft_model = AutoPeftModelForSequenceClassification.from_pretrained("./fine_tuned_imdb_distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Evaluate the fine tuned model
accuracy_after_tuning = trainer.evaluate()

In [18]:
accuracy_after_tuning

{'eval_loss': 0.3333251476287842,
 'eval_accuracy': 0.858,
 'eval_runtime': 61.6175,
 'eval_samples_per_second': 8.115,
 'eval_steps_per_second': 0.519,
 'epoch': 2.0}

In [19]:
print("Accuracy before fine tuning: ", accuracy_before_tuning["eval_accuracy"]*100)
print("Accuracy after fine tuning: ", accuracy_after_tuning["eval_accuracy"]*100)

Accuracy before fine tuning:  50.8
Accuracy after fine tuning:  85.8
